# CH16 Dynamic Programming

In [1]:
# Some important notes:
# DP is a general technique for solving optimization, search, and counting problems that can be decomposed into subproblems.
# what makes DP different is that the same subproblem may reoccur. Therefore, a key to making DP efficient is caching the results of intermediate computations
# The key to solving a DP problem efficiently is finding a way to break the problem into subproblems such that:
# - the original problem can be solved relatively easily once solutions to the subproblems are available, and
# - these subproblem solutions are cached.
# Top tips for DP:
# - Use DP whenever we have to make choices to arrive at the solution. Specifically, when you construct solution to the given instance from solutions to subinstances of smaller problems of the same kind.
# - Apart from optimization problems, DP is also applicable to counting and decision problems
# - Although conceptually DP involves recursion, often for efficiency the cache is built "bottom up", i.e., iteratively. 
# -- when DP is implemented recursively, the cache data structure is hash table or BST
# -- when DP is implemented iteratively, the cache data structure is one or multi-dimensional array
# - DP is based on combining solutions to subproblems to yield a solution to the original problem which might not work for all problems

### Fibonacci Series

In [9]:
# Fibonacci Series Time Complexity: O(N) Space Complexity: O(N)
def fibonacci(n, cache = {}):
    if n <= 1:
        return n
    elif n not in cache.keys():
        cache[n] = fibonacci(n-1) + fibonacci(n-2)
    return cache[n]

print(fibonacci(7))

# Time Complexity: O(N) Space Complexity: O(1)
def fibonacci_space_opt(n):
    if n <= 1:
        return n
    
    f_minus_2, f_minus_1 = 0, 1
    for _ in range(1, n):
        f = f_minus_2 + f_minus_1
        f_minus_2, f_minus_1 = f_minus_1, f
    return f_minus_1 # as f is stored in f_minus_1 in the last step

print(fibonacci_space_opt(7))

13
13


### Maximum sum over all subarrays

In [13]:
# Task: Find the maximum sum over all subarrays of a given array of integer. The array can have positive and negative integers.
# we need to find continous elements with maximum sum. 
# - If all the elements of the array are positive => max_sum = sum of all elements of the array
# - If all the elements of the array are negative => max_sum = max element of the array

# Brute Force: we can form all sub arrays and find each sum and then find max using two for loops. First, subarrays of size 1 then subarrays of size 2 etc.
# Total number of subarrays: ((n*(n+1))/2) and time to compute sum of each subarray is O(n)=>Time Complexity:O(n^3)
def max_sum_subarray_brute_force(a):
    ans = float('-inf')
    
    for sub_array_size in range(1,len(a) + 1):
        for start_index in range(0, len(a)):
            if (start_index + sub_array_size) > len(a): # subarray exceeds array bounds
                break
            sum_val = 0
            for i in range(start_index, start_index + sub_array_size):
                sum_val += a[i]
            ans = max(ans, sum_val)
    return ans
print(f'Brute Force:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')
a = [-2, -5, 6, -2, -3, 1, 5, -6]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')

# Brute force algo can be improved by first computing S[k]=sigma(A[0,k]) for all k. Then sum for A[i,j] is then S[j] - S[i-1]. Time complexity can be reduced to O(n^2) in this method.
def max_sum_subarray_brute_force_opt(a):
    ans = float('-inf')
    sum_till_i = []
    sum_till_i[0] = a[0]
    for i in range(1, len(a)): # sub array sum till k is computed
        sum_till_i[i] += sum_till_i[i-1] + a[i]
    
    # Generating all sub arrays and finding their sum
    for sub_array_size in range(1, len(a) + 1):
        for start_index in range(0, len(a)):
            if start_index + sub_array_size > len(a):
                break
            end_index = start_index + sub_array_size - 1
            sum_val = sum_till_i[end_index] - sum_till_i[start_index-1]
            ans = max(ans, sum_val)
    return ans
print(f'\nBrute Force Optimized:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')
a = [-2, -5, 6, -2, -3, 1, 5, -6]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')

# Divide and conquer: The approach is similar to quick sort. Split array into two halves => compute max sum in left and right=> max subarray sum is going to be max(l+r)
# Time Complexity: O(nlogn)
def max_sum_subarray_divide_and_conquer(a):
    def max_sum_subarray_divide_and_conquer_helper(a, start, end):
        if start == end:
            return a[start]
        
        m = (start + ((end - start)//2))
        sum_l = max_sum_subarray_divide_and_conquer_helper(a, 0, m)
        sum_r = max_sum_subarray_divide_and_conquer_helper(a, m+1, end)
        ans = max(ans, sum_l + sum_r)
        return ans
    
    return max_sum_subarray_divide_and_conquer_helper(a, 0, (len(a) - 1))

print(f'\nDivide and Conquer:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')
a = [-2, -5, 6, -2, -3, 1, 5, -6]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')

# Kadane's Algorithm: The approach uses dynamic programming and its time complexity is O(n) and space complxity is O(1).
# Ref: https://www.baeldung.com/java-maximum-subarray
# Formula: maximumSubArraySum[i] = maximumSubArraySum[i-1] + arr[i] if arr[i] < maximumSubArraySum[i-1] + arr[i] else maximumSubArraySum[i] = arr[i]
def max_sum_subarray_DP(a):
    max_so_far = float('-inf')
    max_ending_here = float('-inf')
    for i in range(0, len(a)):
        if a[i] > max_ending_here + a[i]: # starting a new sub array
            start = i
            max_ending_here = a[i]
        else:
            max_ending_here = max_ending_here + a[i] # adding element to the earlier sub array
        
        if max_so_far < max_ending_here: # updating global maximum
            max_so_far = max_ending_here
            end = i
    print(f'Found maximum sum between {start} and {end} and the maximum sum is {max_so_far}')
    return
print(f'\nKadanes Algorithm DP:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
max_sum_subarray_DP(a)
a = [-2, -5, 6, -2, -3, 1, 5, -6]
max_sum_subarray_DP(a)

Brute Force:
The max sum is 1479
The max sum is 7

Brute Force Optimized:
The max sum is 1479
The max sum is 7

Divide and Conquer:
The max sum is 1479
The max sum is 7

Kadanes Algorithm DP:
Found maximum sum between 0 and 3 and the maximum sum is 1479
Found maximum sum between 2 and 6 and the maximum sum is 7


## 16.1 Count the numeber of score combinations

In [20]:
# In an American football Bame, a play can lead to 2 points (safety), 3 points (field goal), or 7 points (touchdown, assuming the extra point). 
# Many different combinations of 2,3, and 7 point plays can make up a final score. For example, four combinations of plays yield a score of 12: o 6safeties (2x6=12), o 3 safeties and 2 field goals (2 x 3 + 3 x2 = 12), o 1 safety,l field goal and l touchdown (2 x 1+ 3 x 1 +7 x1= 12), and o 4fieldgoals(3x4=12).
# Write a program that takes a final score and scores for individual plays, and returns the number of combinations of plays that result in the final score.

# Generalizing an s-point score can be achieved by an s-2 point score, followed by a 2 point play etc.
# Brute Force: Enumerate all the sequences and count the distinct combinations within these sequences e.g. by sorting each sequence and inserting into a hash table
# Recursion: num_of_combinations[s][n] = num_of_combinations[s-1][n] + num_of_combinations[s][n-s]
# DP : This problem is similar to coin change(https://www.geeksforgeeks.org/coin-change-dp-7/). 
# - Make a table with [0,total_score] cols and 3 rows for 2,3,7 point plays.
# - Compute number of combinations possible keeping a 2point play and not keeping a 2 point play in the sequence etc. Then, add both values to get the total number of possible combinations
# - we find the number of combinations => order is not important

# Time Complexity: O(sn) where s is the len(individual_play_scores) and n = total_score + 1. Space Complexity=O(sn) which is used by the 2D array to store intermediate results
def num_combinations_for_final_score(final_score, individual_play_scores):
    # There is only one way to reach a final_score of 0. so, all the values in the first col are set to 0
    num_combinations_for_score = [([1] + [0]*final_score) for _ in range(len(individual_play_scores))]
    
    for i in range(len(individual_play_scores)):
        for j in range(1, final_score + 1): # compute for one row
            num_combinations_without_this_play = num_combinations_for_score[i-1][j] if i >= 1 else 0
            num_combinations_with_this_play = num_combinations_for_score[i][j - individual_play_scores[i]] if j >= individual_play_scores[i]  else 0
            num_combinations_for_score[i][j] = num_combinations_without_this_play + num_combinations_with_this_play
    return num_combinations_for_score[-1][-1]

individual_play_scores = [2, 3, 7]
final_score = 12
print(f'The total number of possible combinations are {num_combinations_for_final_score(final_score, individual_play_scores)}')

# variant: Solve the same problem using O(n) space
def num_combinations_for_final_score_space_opt(final_score, individual_play_scores):
    # There is only one way to reach a final_score of 0. so, the first value is set to 1
    num_combinations_for_score = ([1] + [0]*final_score)
    
    # Pick all the scores one by one after the index greater than or equal to the values of the picked score
    for i in range(len(individual_play_scores)):# by the time iteration completes, the 1D array is going to be similar to the ith row in the above 2D-array.
        for j in range(individual_play_scores[i], final_score + 1): # compute for one row
            #print(f'i = {i} j = {j} table:{num_combinations_for_score}')
            num_combinations_for_score[j] += num_combinations_for_score[j-individual_play_scores[i]]
            #print(f'i = {i} j = {j} table:{num_combinations_for_score}')
            
    return num_combinations_for_score[-1]
individual_play_scores = [2, 3, 7]
final_score = 12
print(f'Space Opt: The total number of possible combinations are {num_combinations_for_final_score_space_opt(final_score, individual_play_scores)}')

The total number of possible combinations are 4
i = 0 j = 2 table:[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 2 table:[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 3 table:[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 3 table:[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 4 table:[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 4 table:[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 5 table:[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 5 table:[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 6 table:[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
i = 0 j = 6 table:[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
i = 0 j = 7 table:[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
i = 0 j = 7 table:[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
i = 0 j = 8 table:[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
i = 0 j = 8 table:[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
i = 0 j = 9 table:[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
i = 0 j = 9 table:[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
i = 0 j = 10 table:[1, 0

In [36]:
# variant: Write a program that takes a final score and scores for individual plays, and returns the number of sequences of plays that result in the final score. 
# For example, 18 sequences of plays yield a score of 12. Some examples are (2,2,2,3,3), <2,3,2,2,3>, <2,3,7), <7,3,2>.
# This problem is asking for permutations 
# Ref: https://massivealgorithms.blogspot.com/2014/09/count-number-of-score-combinations-epi.html
def num_permutations_for_final_score_space_opt(final_score, individual_play_scores):
    # There is only one way to reach a final_score of 0. So, the first value is set to 1.
    num_permutations_for_score = ([1] + [0]*final_score)
    
    # reverse the order of the loops to get the permutations
    for j in range(0, final_score + 1):
        for i in range(len(individual_play_scores)):
            #print(f'i = {i} j = {j} table:{num_permutations_for_score}')
            if j >= individual_play_scores[i]:
                num_permutations_for_score[j] += num_permutations_for_score[j-individual_play_scores[i]]
                #print(f'i = {i} j = {j} table:{num_permutations_for_score}')
            
    return num_permutations_for_score[-1]

individual_play_scores = [2, 3, 7]
final_score = 12
print(f'Space Opt: The total number of possible permutations are {num_permutations_for_final_score_space_opt(final_score, individual_play_scores)}\n')

# variant: You are climbing stairs. You can advance 1 to k steps at a time. Your destination is exactly n steps up. 
# Write a program which takes as inputs n and k and returns the number of ways in which you can get to your destination. 
# For example, if n = 4 and k = 2, there are five ways in which to get to the destination: 
# - o four single stair advances, 
# - o two single stair advances followed by a double stair advance, 
# - o a single stair advance followed by a double stair advance followed by a single stair advance, 
# - o a double stair advance followed by two single stairs advances, and 
# - o two double stair advances.

# The main difference between score and this problem is : score asks for number of combinations whereas steps asks for number of permutations

k = 2
n = 4
individual_play_scores = [k for k in range(1, k + 1)]
final_score = n
print(f'Steps: Space Opt: The total number of possible permutations are {num_permutations_for_final_score_space_opt(final_score, individual_play_scores)}')

Space Opt: The total number of possible permutations are 18

Steps: Space Opt: The total number of possible permutations are 5


In [38]:
# variant:  Suppose the final score is given in the form (s,s'), i.e., Team 1 scored s points and Team 2 scored s' points. 
# How would you compute the number of distinct scoring sequences which result in this score? 
# For example, if the final score is (6,3) then Team 1 scores 3, Team 2 scores 3, Team 1 scores 3 is a scoring sequence which results in this score.
individual_play_scores = [2, 3, 7]
final_score = 6
num_of_permutations_team1 = num_permutations_for_final_score_space_opt(final_score, individual_play_scores)
print(f'Total permutations for Team1: {num_of_permutations_team1}')
individual_play_scores = [2, 3, 7]
final_score = 3
num_of_permutations_team2 = num_permutations_for_final_score_space_opt(final_score, individual_play_scores)
print(f'Total permutations for Team2: {num_of_permutations_team2}')
total_permutations_together = num_of_permutations_team1 * num_of_permutations_team2
print(f'Total permutations for Team1 and Team2 together: {total_permutations_together}')

Total permutations for Team1: 2
Total permutations for Team2: 1
Total permutations for Team1 and Team2 together: 2


## 16.2 Compute the levenshtein distance

In [42]:
# Spell checkers make suggestions for misspelled words. Given a misspelled string a spell checker should return words in the dictionary which are close to the misspelled string. 
# The number of edits needed to transform one word to another is called as levenshtein distance

# Task: Write a Program that takes two strings and computes the minimum number of edits needed to transform the first string into the second string.

# Brute Force: Enumerate all strings that are at distance 1,2,3 etc and stop once we get the second string. Time complexity: Exponential

# A better approach is to "prune" the search. For example, if the last character of the first string equals the last character of the second string, we can ignore this character. 
# - If they are different, we can focus on the initial portion of each string and perform a final edit step. (This final edit step may be an insertion, deletion, or substitution.)
# Let E(A[0,a - 1]),B[0,b - 1]) be the Levenshtein distance between the strings A and B. Then, the following two cases are possible:
# - If last char of A equal last char of B, then E(A[0,a - 1]),B[0,b - 1]) = E(A[0,a - 2]),B[0,b - 2])
# - else E(A[0,a - 1]),B[0,b - 1]) = 1 + min(E(A[0,a - 2]),B[0,b - 2]), E(A[0,a - 1]),B[0,b - 2]), E(A[0,a - 2]),B[0,b - 1])). Transforming A[0,a - 1] to B[0,b - 1]:
# ---by transforming A[0,a - 2] to B[0,b - 2] and then substituting A's last character with B's last character. 
# ---by transforming A[0,a - 1] to B[0,b - 2] and then adding B's last character at the end. 
# ---by transforming A[0,a - 2] to B[0,b - 1] and then deleting A's last character.

# Time Complexity: O(ab) Space Complexity: O(ab)
def levenshtein_distance(A, B):
    def compute_distance_between_prefixes(A_idx, B_idx):
        if A_idx < 0: 
            return B_idx + 1 # A is empty so add all of B's characters.
        elif B_idx < 0:
            return A_idx + 1 # B is empty so delete all of A's Charcters
        
        if distance_between_prefixes[A_idx][B_idx] == -1: # check not to compute same value multiple times
            if A[A_idx] == B[B_idx]: # last chars of both strings are equal
                distance_between_prefixes[A_idx][B_idx] = compute_distance_between_prefixes(A_idx - 1, B_idx - 1)
            else: # finding min of substitution, add, delete
                substitute_last = compute_distance_between_prefixes(A_idx - 1, B_idx - 1)
                add_last = compute_distance_between_prefixes(A_idx - 1, B_idx)
                delete_last = compute_distance_between_prefixes(A_idx, B_idx - 1)
                distance_between_prefixes[A_idx][B_idx] = (1 + min(substitute_last, add_last, delete_last))
        return distance_between_prefixes[A_idx][B_idx]
        
    distance_between_prefixes = [[-1] * len(B) for _ in A]
    return compute_distance_between_prefixes(len(A) - 1, len(B) - 1)

A = "Carthorse"
B = "Orchestra"
print(f'The levenshtein distance between {A} and {B} is {levenshtein_distance(A, B)}')       
    

The levenshtein distance between Carthorse and Orchestra is 8


In [46]:
# Variant: Compute the Levenshtein distance using O(min(a, b)) space and O(ab) time.
# Generally if we are filling the i = 10 rows in DP we require only values of 9th row. 
# Ref: https://www.geeksforgeeks.org/edit-distance-dp-5/

# Create table with two rows: use (A_idx % 2) while storing values into the table
# Time Complexity: O(ab) Space Complexity: O(b)
def levenshtein_distance_space_opt(A, B):
    def compute_distance_between_prefixes(A_idx, B_idx):
        if A_idx < 0: 
            return B_idx + 1 # A is empty so add all of B's characters.
        elif B_idx < 0:
            return A_idx + 1 # B is empty so delete all of A's Charcters
        
        if distance_between_prefixes[(A_idx%2)][B_idx] == -1:# check not to compute same value multiple times
            if A[A_idx] == B[B_idx]: # last chars of both strings are equal
                distance_between_prefixes[(A_idx%2)][B_idx] = compute_distance_between_prefixes(A_idx - 1, B_idx - 1)
            else: # finding min of substitution, add, delete
                substitute_last = compute_distance_between_prefixes(A_idx - 1, B_idx - 1)
                add_last = compute_distance_between_prefixes(A_idx - 1, B_idx)
                delete_last = compute_distance_between_prefixes(A_idx, B_idx - 1)
                distance_between_prefixes[(A_idx%2)][B_idx] = (1 + min(substitute_last, add_last, delete_last))
        return distance_between_prefixes[(A_idx%2)][B_idx]
        
    distance_between_prefixes = [[-1] * len(B) for _ in range(0,2)] # create two rows: one for previous values and one to compute current values
    return compute_distance_between_prefixes(len(A) - 1, len(B) - 1)

A = "Carthorse"
B = "Orchestra"
print(f'The levenshtein distance between {A} and {B} is {levenshtein_distance_space_opt(A, B)}')   

The levenshtein distance between Carthorse and Orchestra is 8


### Variant: Longest Common Subsequence

In [50]:
# variant: Given A and B as above, compute a longest sequence of characters that is a subsequence of A and of B. F
# For example, the longest subsequence which is present in both strings in "Carthorse" and "Orchestra" is (r,h,s).

# Ref: https://www.geeksforgeeks.org/longest-common-subsequence-dp-4/
# A subsequence is a sequence that appears in the same relative order, but not necessarily contiguous.
# For ex: "abc", "abg", "bdf", "aeg", "acefg", ... etc are subsequences of "abcdefg"
# LCS for input Sequences “ABCDGH” and “AEDFHR” is “ADH” of length 3.
# LCS for input Sequences “AGGTAB” and “GXTXAYB” is “GTAB” of length 4.

# Brute Force: Number of subsequence of lenght 1,2,...,n is nC0 + nC1 + nC2 + ... + nCn = 2^n and O(n) to check if subsequence is common to both strings => Time Complexity: O(n*(2^n))

# DP Approach: For two string (A, B) let L(A[0,a-1],B[0,b-1]) be the length of longest common subsequence. we get the following two cases:
# - If last char of A equal last char of B, then L(A[0,a-1], B[0, b-1]) = 1 + L(A[0,a-2], B[0, b-2])
# - else L(A[0,a-1], B[0, b-1]) = max(L(A[0,a-2], B[0, b-1]), L(A[0,a-1], B[0, b-2]))
# Time Complexity: O(ab) Space Complexity: O(ab)
def longest_common_subsequence(A, B):
    def compute_lcs_between_prefixes(A_idx, B_idx):
        if A_idx < 0:
            return 0
        elif B_idx < 0:
            return 0
        
        if lcs_between_prefixes[A_idx][B_idx] == -1: # check not to compute same value multiple times
            if(A[A_idx] == B[B_idx]):
                lcs_between_prefixes[A_idx][B_idx] = 1 + compute_lcs_between_prefixes(A_idx - 1, B_idx - 1)
            else:
                lcs_between_prefixes[A_idx][B_idx] = max(compute_lcs_between_prefixes(A_idx, B_idx - 1), compute_lcs_between_prefixes(A_idx - 1, B_idx))
        return lcs_between_prefixes[A_idx][B_idx]
    
    lcs_between_prefixes = [[-1] * len(B) for _ in A]
    return compute_lcs_between_prefixes(len(A) - 1, len(B) - 1)

A = "Carthorse"
B = "Orchestra"
print(f'The lenght of longest common subsequence between {A} and {B} is {longest_common_subsequence(A, B)}') 
A = "AGGTAB"
B = "GXTXAYB"
print(f'The lenght of longest common subsequence between {A} and {B} is {longest_common_subsequence(A, B)}') 

The lenght of longest common subsequence between Carthorse and Orchestra is 3
The lenght of longest common subsequence between AGGTAB and GXTXAYB is 4


### Variant: Min number of deletions to convert string to palindrome

In [64]:
# Variant: Given a string A, compute teh minimum number of characters you need to delete from A to make the resulting string a palindrome
# Ref: https://www.geeksforgeeks.org/minimum-number-deletions-make-string-palindrome/

# Brute Force: Remove all subseqences one by one and check if the string is a palindrome Time Complexity:O(n*2^n)

# DP Approach: An efficient approach uses the concept of finding the length of the longest palindromic subsequence of a given sequence.

# DP to compute length of longest palindromic subsequence: https://www.geeksforgeeks.org/longest-palindromic-subsequence-dp-12/
# For a string, let LPS(A[0,a-1]) be the length of longest palindromic subsequence. we get the following two cases:
# - If first char of A equal to last char of A, then LPS(A[0,a-1]) = 2 + LPS(A[1,a-2])
# - else LPS(A[0,a-1]) = max(L(A[1,a-1]), L(A[0,a-2]))
# Time Complexity: O(a^2) Space Complexity: O(a^2)
def longest_palindromic_subsequence(A):
    def compute_lps_between_prefixes(A_idx_start, A_idx_end):
        if A_idx_start > A_idx_end:
            return 0
        elif A_idx_start == A_idx_end: # string A is of odd lenght
            return 1
        elif A_idx_end < 0:
            return 0
        
        if lps_between_prefixes[A_idx_start][A_idx_end] == -1: # check not to compute same value multiple times
            if(A[A_idx_start] == A[A_idx_end]):
                lps_between_prefixes[A_idx_start][A_idx_end] = 2 + compute_lps_between_prefixes(A_idx_start + 1, A_idx_end - 1)
            else:
                lps_between_prefixes[A_idx_start][A_idx_end] = max(compute_lps_between_prefixes(A_idx_start + 1, A_idx_end), compute_lps_between_prefixes(A_idx_start, A_idx_end - 1))
        return lps_between_prefixes[A_idx_start][A_idx_end]
    
    lps_between_prefixes = [[-1] * len(A) for _ in A]
    return compute_lps_between_prefixes(0, len(A) - 1)

A = "GEEKSFORGEEKS"
lps = longest_palindromic_subsequence(A)
min_num_of_deletions = len(A) - lps
print(f'The length of the longest palindromic subsequence in {A} is {lps}')
print(f'Minimum number of deletions requried to convert {A} to palindrome are {min_num_of_deletions}\n')
A = "aebbda"
lps = longest_palindromic_subsequence(A)
min_num_of_deletions = len(A) - lps
print(f'The length of the longest palindromic subsequence in {A} is {lps}')
print(f'Minimum number of deletions requried to convert {A} to palindrome are {min_num_of_deletions}')

The length of the longest palindromic subsequence in GEEKSFORGEEKS is 5
Minimum number of deletions requried to convert GEEKSFORGEEKS to palindrome are 8

The length of the longest palindromic subsequence in aebbda is 4
Minimum number of deletions requried to convert aebbda to palindrome are 2
